### Methode :
- **Support Vector Machine (SVM)** : Un algorithme de classification qui trouve l'hyperplan optimal dans un espace de grande dimension pour séparer les différentes classes. Il peut également être étendu pour gérer des problèmes non linéaires en utilisant des noyaux.

- **Régression logistique** : Un algorithme utilisé pour la classification binaire (et pouvant être étendu à la classification multiclasse) en modélisant la probabilité que chaque classe soit la classe cible à l'aide d'une fonction logistique.
- **Random Forest** : Un algorithme d'ensemble utilisé pour la classification et la régression. Il combine les prédictions de plusieurs arbres de décision pour obtenir une prédiction plus robuste et généralement de meilleure qualité.
- **Réseaux de neurones** :
- **Perceptron ou Multi-perceptron** :
- **Gradient Boosting** : Un autre algorithme d'ensemble qui construit des arbres de décision de manière séquentielle, en corrigeant les erreurs des arbres précédents. Cela conduit à un modèle de prédiction puissant.
- **Naive Bayes** : Un classificateur probabiliste simple basé sur le théorème de Bayes avec une forte indépendance entre les fonctionnalités. Il est souvent utilisé pour la classification de texte et d'autres tâches où l'indépendance des fonctionnalités est une hypothèse raisonnable.





In [10]:

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder ,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle



In [11]:
# Load dataset
data_dir_path = '../src/data/'
data_trainset_cleaned = pd.read_csv(data_dir_path + "/data_trainset_cleaned.csv")

In [12]:
# Create a copy to avoid modifying the original DataFrame
dataset_encoded = data_trainset_cleaned.copy()

# Encoding the 'species' column
encoder = LabelEncoder()
dataset_encoded['species'] = encoder.fit_transform(dataset_encoded['species'])

# Select numeric columns except for the already encoded 'species' column for imputation
numeric_columns = dataset_encoded.select_dtypes(include=['int64', 'float64']).columns

# Initialize the KNNImputer instance
imputer = KNNImputer(n_neighbors=5)

# Apply imputation only on the numeric columns
dataset_encoded[numeric_columns] = imputer.fit_transform(dataset_encoded[numeric_columns])

# Convert the NumPy array returned by KNNImputer back into a pandas DataFrame and match column names
trainset_imputed = pd.DataFrame(dataset_encoded, columns=dataset_encoded.columns)

# Calculate the percentage of NaN values for each column after imputation
na_percentage = trainset_imputed.isna().mean() * 100

# Display the percentages
print("Percentage of missing data per column after imputation:")
print(na_percentage)

Percentage of missing data per column after imputation:
species      0.0
margin1      0.0
margin2      0.0
margin3      0.0
margin4      0.0
            ... 
texture60    0.0
texture61    0.0
texture62    0.0
texture63    0.0
texture64    0.0
Length: 187, dtype: float64


# Mixing and Splitting of Data 

In [13]:
# Shuffle the new dataset
new_dataset_shuffled = shuffle(trainset_imputed)

# Separate target from data
features = trainset_imputed.drop("species", axis=1)
new_target = trainset_imputed['species']

# Create training set and testing set
features_train, features_test, target_train, target_test = train_test_split(
    features, new_target, test_size=0.2, stratify=new_target
)

# Standardization of the test set
dataset_standardized = features_test.copy()
scaler = StandardScaler()
dataset_standardized[features_test.columns] = scaler.fit_transform(dataset_standardized[features_test.columns])

features_test_standardized = dataset_standardized


# Entrainement

-------------------------------------------------------
